In [ ]:
## astrocyte analysis round 1

import numpy as np
import matplotlib.pyplot as plt
import tifffile
import glob
import pandas as pd

import os

from skimage.morphology import erosion, ball

# Set the working directory
os.chdir('/Users/katherineridley/Projects/PlaqueStack/OC_6E10/')


# Define file patterns


gfap_files = sorted(glob.glob('*GFAP.tiff'))
nuclei_files = sorted(glob.glob('*Nuclei.tiff'))
oc_files = sorted(glob.glob('*OC_combined.tiff'))
e6_files = sorted(glob.glob('*6E10_combined.tiff'))

# Ensure that the files are matched correctly
file_pairs = list(zip(gfap_files, nuclei_files, oc_files, e6_files))

def pad_image_to_target_size(image, target_shape):
    original_shape = image.shape
    pad_depth = target_shape[0] - original_shape[0]
    pad_height = target_shape[1] - original_shape[1]
    pad_width = target_shape[2] - original_shape[2]
    
    # Depth padding
    pad_depth_before = pad_depth // 2 if pad_depth > 0 else 0
    pad_depth_after = pad_depth - pad_depth_before if pad_depth > 0 else 0
    
    # Height padding
    pad_top = pad_height // 2 if pad_height > 0 else 0
    pad_bottom = pad_height - pad_top if pad_height > 0 else 0
    
    # Width padding
    pad_left = pad_width // 2 if pad_width > 0 else 0
    pad_right = pad_width - pad_left if pad_width > 0 else 0
    
    # Create the padding tuple
    padding = (
        (pad_depth_before, pad_depth_after),
        (pad_top, pad_bottom),
        (pad_left, pad_right)
    )
    
    # Pad the image
    padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    
    return padded_image

def plot_max_projection(image, title):
    max_proj = np.max(image, axis=0)
    plt.figure(figsize=(8, 8))
    #cmap color

    plt.imshow(max_proj, cmap='viridis')
    plt.title(title)
    plt.axis('off')
    plt.show()

from skimage.measure import label

def segment_nuclei(nuclei_img):
    
    labeled_nuclei = label(nuclei_img)
    return labeled_nuclei


def get_nuclei_touching_plaque(labeled_nuclei, plaque_mask):
    from skimage.measure import regionprops
    nuclei_props = regionprops(labeled_nuclei)
    nuclei_to_remove = []

    for prop in nuclei_props:
        label_id = prop.label
        nucleus_coords = prop.coords  # Coordinates of the nucleus pixels
        # Check if any nucleus pixels overlap with the plaque
        overlapping = plaque_mask[tuple(zip(*nucleus_coords))].any()
        if overlapping:
            nuclei_to_remove.append(label_id)
    return nuclei_to_remove

def remove_nuclei_touching_plaque(labeled_nuclei, nuclei_to_remove):
    # Create a mask of nuclei to keep
    nuclei_mask = np.isin(labeled_nuclei, nuclei_to_remove, invert=True) & (labeled_nuclei > 0)
    # Relabel the nuclei
    labeled_nuclei_clean = label(nuclei_mask)
    return labeled_nuclei_clean

from skimage.measure import regionprops_table
import pandas as pd

def measure_nuclei_volumes(labeled_nuclei):
    """
    Measures the volume of each nucleus.

    Parameters:
    - labeled_nuclei: 3D NumPy array of labeled nuclei.

    Returns:
    - nuclei_df: DataFrame containing nucleus labels and volumes.
    """
    props = regionprops_table(labeled_nuclei, properties=['label', 'area'])
    nuclei_df = pd.DataFrame(props)
    nuclei_df.rename(columns={'area': 'volume'}, inplace=True)
    return nuclei_df


def get_volume_percentiles(nuclei_df):
    """
    Calculates the 10th and 90th percentiles of nuclei volumes.

    Parameters:
    - nuclei_df: DataFrame containing nucleus labels and volumes.

    Returns:
    - volume_min: 10th percentile volume.
    - volume_max: 90th percentile volume.
    """
    volume_min = nuclei_df['volume'].quantile(0.20)
    volume_max = nuclei_df['volume'].quantile(0.99)
    return volume_min, volume_max

def filter_nuclei_by_volume(nuclei_df, volume_min, volume_max):
    """
    Filters nuclei within the specified volume range.

    Parameters:
    - nuclei_df: DataFrame containing nucleus labels and volumes.
    - volume_min: Minimum volume threshold.
    - volume_max: Maximum volume threshold.

    Returns:
    - valid_labels: List of nucleus labels within the volume range.
    """
    valid_nuclei = nuclei_df[(nuclei_df['volume'] >= volume_min) & (nuclei_df['volume'] <= volume_max)]
    valid_labels = valid_nuclei['label'].tolist()
    return valid_labels

def remove_nuclei_outside_volume_range(labeled_nuclei, valid_labels):
    """
    Removes nuclei outside the specified volume range.

    Parameters:
    - labeled_nuclei: 3D NumPy array of labeled nuclei.
    - valid_labels: List of nucleus labels to keep.

    Returns:
    - labeled_nuclei_filtered: 3D NumPy array of filtered nuclei labels.
    """
    import numpy as np
    nuclei_mask = np.isin(labeled_nuclei, valid_labels)
    labeled_nuclei_filtered = label(nuclei_mask)
    return labeled_nuclei_filtered


def segment_oc_objects(oc_img):
    """
    Segments the OC image to create a binary mask of OC regions.

    Parameters:
    - oc_img: 3D NumPy array of the OC image.

    Returns:
    - oc_mask: 3D NumPy array (boolean) of the OC mask.
    """
    # Apply Otsu's thresholding
    thresh = threshold_otsu(oc_img)
    oc_mask = oc_img > thresh

    # Remove small objects (optional)
    #oc_mask = remove_small_objects(oc_mask, min_size=100)

    #make oc_mask smaller
    #oc_mask = erosion(oc_mask, ball(1))

    return oc_mask


from skimage.filters import sobel
from skimage import img_as_float

def prepare_gfap_for_watershed(gfap_img):
    # Convert the GFAP image to float
    gfap_float = img_as_float(gfap_img)
    # Invert the GFAP image
    gfap_inverted = gfap_float.max() - gfap_float
    # Compute the gradient
    gradient = sobel(gfap_inverted)
    return gradient

from skimage.segmentation import watershed

def segment_astrocytes_watershed(gfap_img, markers):
    gradient = prepare_gfap_for_watershed(gfap_img)
    # Apply watershed
    labeled_astrocytes = watershed(gradient, markers, mask=gfap_img > 0)
    return labeled_astrocytes



from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage as ndi

import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, closing, ball
from skimage.measure import label, regionprops
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy.ndimage import binary_dilation
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, closing, ball
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.measure import label, regionprops_table
from scipy import ndimage as ndi

def segment_gfap(gfap_img, labeled_nuclei_filtered, oc_mask, voxel_size=(1.0, 0.18, 0.18)):
    """
    Segments the GFAP image using the watershed algorithm after removing pixels overlapping with OC objects.

    Parameters:
    - gfap_img: 3D NumPy array of the GFAP image.
    - oc_mask: 3D NumPy array (boolean) of the OC mask.
    - labeled_nuclei_filtered: 3D NumPy array of labeled nuclei used as markers.
    - voxel_size: tuple of floats, voxel dimensions in micrometers (depth, height, width).

    Returns:
    - labeled_gfap: 3D NumPy array with labeled GFAP regions.
    """
    import numpy as np
    from skimage.filters import threshold_otsu
    from skimage.morphology import remove_small_objects, closing, ball
    from skimage.segmentation import watershed
    from scipy import ndimage as ndi
    from skimage.measure import label

    # Apply Otsu's thresholding
    thresh = threshold_otsu(gfap_img)
    gfap_mask = gfap_img > thresh

    # Remove small objects
    gfap_mask = remove_small_objects(gfap_mask, min_size=10)

    # Close objects
    gfap_mask = closing(gfap_mask, ball(1))

    # Remove overlapping pixels with OC mask
    gfap_mask = gfap_mask & (~oc_mask)

    #plot max projection of gfap mask
    max_proj = np.max(gfap_mask, axis=0)
    plt.figure(figsize=(8, 8))
    #cmap color

    plt.imshow(max_proj, cmap='viridis')
    plt.title('GFAP Mask')
    plt.axis('off')
    plt.show()

    # Proceed only if there are any remaining GFAP pixels
    if np.sum(gfap_mask) == 0:
        print("GFAP mask is empty after removing overlapping pixels with OC.")
        return np.zeros_like(gfap_mask, dtype=int)

    # Compute the distance transform
    distance = ndi.distance_transform_edt(gfap_mask)

    # Use labeled nuclei as markers
    markers = labeled_nuclei_filtered

    # Apply the watershed algorithm
    gradient = prepare_gfap_for_watershed(gfap_mask)
    labeled_gfap = watershed(gradient, markers, mask=gfap_mask>0)

    labeled_gfap = label(labeled_gfap)

    # Merge small segments (if needed)
    #labeled_gfap = merge_small_gfap_segments(labeled_gfap, voxel_size)

    return labeled_gfap


def remove_gfap_segments_overlapping_oc(labeled_gfap, oc_mask):
    """
    Removes GFAP segments that overlap with OC objects.

    Parameters:
    - labeled_gfap: 3D NumPy array with labeled GFAP regions.
    - oc_mask: 3D NumPy array (boolean) of the OC mask.

    Returns:
    - labeled_gfap_cleaned: 3D NumPy array with overlapping GFAP segments removed.
    """
    import numpy as np

    # Find labels of GFAP segments that overlap with OC mask
    overlapping_labels = np.unique(labeled_gfap[oc_mask])

    # Remove background label (0) if present
    overlapping_labels = overlapping_labels[overlapping_labels != 0]

    # Create a mask of GFAP segments to keep
    gfap_mask_to_keep = ~np.isin(labeled_gfap, overlapping_labels) & (labeled_gfap > 0)

    # Relabel the remaining GFAP segments
    labeled_gfap_cleaned = label(gfap_mask_to_keep)

    return labeled_gfap_cleaned



def merge_small_gfap_segments(labeled_gfap, voxel_size, volume_threshold=10):
    """
    Merges small GFAP segments with neighboring astrocytes.

    Parameters:
    - labeled_gfap: 3D NumPy array with labeled GFAP regions.
    - voxel_size: tuple of floats, voxel dimensions in micrometers (depth, height, width).
    - volume_threshold: float, minimum volume (in μm³) for GFAP segments.

    Returns:
    - labeled_gfap_merged: 3D NumPy array with updated labels.
    """
    from skimage.morphology import dilation, ball

    # Calculate voxel volume
    voxel_volume = voxel_size[0] * voxel_size[1] * voxel_size[2]  # μm³ per voxel

    # Get properties of GFAP segments
    props = regionprops_table(labeled_gfap, properties=['label', 'area'])
    gfap_df = pd.DataFrame(props)
    gfap_df['volume'] = gfap_df['area'] * voxel_volume

    # Identify small segments
    small_segments = gfap_df[gfap_df['volume'] < volume_threshold]['label'].tolist()

    # Create a copy of labeled_gfap to modify
    labeled_gfap_merged = labeled_gfap.copy()

    # Create a mapping of labels to merge
    label_mapping = {}

    for label_id in small_segments:
        # Create a mask for the small segment
        small_segment_mask = labeled_gfap == label_id

        # Dilate the small segment to find neighbors
        dilated_mask = dilation(small_segment_mask, ball(1))

        # Find neighboring labels
        neighbor_labels = np.unique(labeled_gfap[dilated_mask])
        neighbor_labels = neighbor_labels[(neighbor_labels != 0) & (neighbor_labels != label_id)]

        if len(neighbor_labels) > 0:
            # Find the largest neighboring segment
            neighbor_volumes = gfap_df[gfap_df['label'].isin(neighbor_labels)].set_index('label')['volume']
            largest_neighbor_label = neighbor_volumes.idxmax()

            # Map the small segment label to the largest neighbor label
            label_mapping[label_id] = largest_neighbor_label
        else:
            # No neighbors; set the label to 0 (remove it)
            label_mapping[label_id] = 0

    # Apply the label mapping
    for small_label, target_label in label_mapping.items():
        labeled_gfap_merged[labeled_gfap == small_label] = target_label

    # Relabel to ensure labels are consecutive
    labeled_gfap_merged = label(labeled_gfap_merged > 0)

    return labeled_gfap_merged

from skimage.measure import regionprops_table
import pandas as pd

def measure_astrocyte_volumes(labeled_astrocytes, voxel_size):
    """
    Measures the volume of each astrocyte segment.

    Parameters:
    - labeled_astrocytes: 3D NumPy array of labeled astrocyte segments.
    - voxel_size: tuple of floats, voxel dimensions in micrometers (depth, height, width).

    Returns:
    - astrocyte_df: DataFrame containing astrocyte labels and volumes.
    """
    props = regionprops_table(labeled_astrocytes, properties=['label', 'area'])
    astrocyte_df = pd.DataFrame(props)
    voxel_volume = voxel_size[0] * voxel_size[1] * voxel_size[2]  # μm³ per voxel
    astrocyte_df['volume'] = astrocyte_df['area'] * voxel_volume
    return astrocyte_df

def get_nuclei_within_segment(labeled_astrocytes, labeled_nuclei, segment_label):
    """
    Identifies nuclei within a given astrocyte segment.

    Parameters:
    - labeled_astrocytes: 3D NumPy array of labeled astrocyte segments.
    - labeled_nuclei: 3D NumPy array of labeled nuclei.
    - segment_label: int, label of the astrocyte segment.

    Returns:
    - nuclei_labels: list of int, labels of nuclei within the astrocyte segment.
    """
    segment_mask = labeled_astrocytes == segment_label
    nuclei_in_segment = labeled_nuclei[segment_mask]
    nuclei_labels = np.unique(nuclei_in_segment)
    nuclei_labels = nuclei_labels[nuclei_labels != 0]  # Exclude background
    return nuclei_labels.tolist()

from skimage.segmentation import watershed

def split_large_segment(astrocyte_segment_mask, nuclei_labels, labeled_nuclei):
    """
    Splits a large astrocyte segment into smaller segments based on nuclei within it.

    Parameters:
    - astrocyte_segment_mask: 3D boolean array, mask of the astrocyte segment.
    - nuclei_labels: list of int, labels of nuclei within the astrocyte segment.
    - labeled_nuclei: 3D NumPy array of labeled nuclei.

    Returns:
    - segmented_astrocyte: 3D NumPy array, labeled segments within the astrocyte segment.
    """
    import numpy as np
    from scipy import ndimage as ndi

    # Create markers for the nuclei within the segment
    markers = np.zeros_like(astrocyte_segment_mask, dtype=int)
    for nucleus_label in nuclei_labels:
        markers[labeled_nuclei == nucleus_label] = nucleus_label

    # Compute the distance transform within the segment
    distance = ndi.distance_transform_edt(astrocyte_segment_mask)

    # Apply watershed segmentation
    segmented_astrocyte = watershed(-distance, markers, mask=astrocyte_segment_mask)

    return segmented_astrocyte

def update_astrocyte_labels(labeled_astrocytes, large_segment_label, segmented_astrocyte):
    """
    Updates the labeled astrocyte image by replacing a large segment with split segments.

    Parameters:
    - labeled_astrocytes: 3D NumPy array of labeled astrocyte segments.
    - large_segment_label: int, label of the large astrocyte segment.
    - segmented_astrocyte: 3D NumPy array, labeled segments within the astrocyte segment.

    Returns:
    - updated_labeled_astrocytes: 3D NumPy array, updated labeled astrocyte image.
    """
    import numpy as np

    # Find the maximum label in the current labeled_astrocytes
    max_label = labeled_astrocytes.max()

    # Relabel segmented_astrocyte to ensure unique labels
    segmented_astrocyte[segmented_astrocyte != 0] += max_label

    # Replace the large segment with the new segments
    updated_labeled_astrocytes = labeled_astrocytes.copy()
    updated_labeled_astrocytes[labeled_astrocytes == large_segment_label] = 0
    updated_labeled_astrocytes += segmented_astrocyte

    return updated_labeled_astrocytes

def split_large_astrocytes(labeled_astrocytes, labeled_nuclei, voxel_size, max_volume=500, min_volume=40):
    """
    Splits large astrocyte segments into smaller segments based on nuclei within them.

    Parameters:
    - labeled_astrocytes: 3D NumPy array of labeled astrocyte segments.
    - labeled_nuclei: 3D NumPy array of labeled nuclei.
    - voxel_size: tuple of floats, voxel dimensions in micrometers (depth, height, width).
    - max_volume: float, maximum volume (in μm³) for astrocyte segments.

    Returns:
    - updated_labeled_astrocytes: 3D NumPy array, updated labeled astrocyte image.
    """
    astrocyte_df = measure_astrocyte_volumes(labeled_astrocytes, voxel_size)

    print(astrocyte_df)
    large_segments = astrocyte_df[(astrocyte_df['volume'] > max_volume) & (astrocyte_df['volume'] < min_volume)]['label'].tolist()

    updated_labeled_astrocytes = labeled_astrocytes.copy()

    for segment_label in large_segments:
        # Get the mask of the large astrocyte segment
        segment_mask = labeled_astrocytes == segment_label

        # Find nuclei within the segment
        nuclei_labels = get_nuclei_within_segment(labeled_astrocytes, labeled_nuclei, segment_label)

        # If no nuclei are found, decide how to handle (e.g., keep the segment as is or remove it)
        if not nuclei_labels:
            print(f"No nuclei found within large segment {segment_label}.")
            continue

        # Split the segment using nuclei as markers
        segmented_astrocyte = split_large_segment(segment_mask, nuclei_labels, labeled_nuclei)

        # Update the labeled astrocyte image
        updated_labeled_astrocytes = update_astrocyte_labels(
            updated_labeled_astrocytes, segment_label, segmented_astrocyte
        )

    # Relabel the updated astrocyte image to ensure labels are consecutive
    updated_labeled_astrocytes = label(updated_labeled_astrocytes > 0)

    return updated_labeled_astrocytes

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, label

def astrocyte_volume(labeled_gfap, voxel_size, percentile=1):
    """
    Removes astrocyte segments in the lowest percentile of volumes.

    Parameters:
    - labeled_astrocytes: 3D NumPy array of labeled astrocyte segments.
    - voxel_size: tuple of floats, voxel dimensions in micrometers (depth, height, width).
    - percentile: float, the percentile below which segments will be removed.

    Returns:
    - labeled_astrocytes_filtered: 3D NumPy array with small segments removed.
    """
    # Calculate voxel volume
    voxel_volume = voxel_size[0] * voxel_size[1] * voxel_size[2]  # μm³ per voxel

    # Measure volumes of astrocyte segments
    props = regionprops_table(labeled_gfap, properties=['label', 'area'])
    astrocyte_df = pd.DataFrame(props)
    astrocyte_df['volume'] = astrocyte_df['area'] * voxel_volume

    print(astrocyte_df['volume'])

    return astrocyte_df

def visualize_astrocytes_max_projection(labeled_astrocytes, title='Astrocyte Segmentation - Max Projection'):
    import numpy as np
    import matplotlib.pyplot as plt
    from skimage.color import label2rgb
    from skimage.measure import regionprops

    # Create a maximum projection along the z-axis
    labels_projection = np.max(labeled_astrocytes, axis=0)

    # Create RGB image
    labels_rgb = label2rgb(labels_projection, bg_label=0)

    # Initialize the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(labels_rgb)
    plt.title(title)
    plt.axis('off')

    # Get region properties
    props = regionprops(labels_projection)

    # Overlay label numbers
    for prop in props:
        y0, x0 = prop.centroid
        plt.text(x0, y0, str(prop.label), color='white', fontsize=12, ha='center', va='center')

    plt.show()





from scipy.ndimage import binary_dilation

def find_astrocytes_touching_nuclei(labeled_gfap, labeled_nuclei):
    # Dilate nuclei to ensure overlap
    dilated_nuclei = binary_dilation(labeled_nuclei, structure=np.ones((3, 3, 3)))
    astrocyte_mask = np.zeros_like(labeled_gfap)
    
    for nucleus_label in np.unique(labeled_nuclei):
        if nucleus_label == 0:
            continue
        nucleus = (labeled_nuclei == nucleus_label)
        overlapping_gfap = labeled_gfap * nucleus
        overlapping_labels = np.unique(overlapping_gfap)
        for label_id in overlapping_labels:
            if label_id == 0:
                continue
            astrocyte_mask[labeled_gfap == label_id] = nucleus_label
    return astrocyte_mask

from skimage.morphology import closing, ball

def close_astrocyte_objects(astrocyte_mask):
    closed_mask = closing(astrocyte_mask > 0, ball(1))
    labeled_astrocytes = label(closed_mask)
    return labeled_astrocytes

from skimage.measure import regionprops_table
from skimage.morphology import skeletonize_3d


import numpy as np
from scipy.ndimage import convolve

from skan import Skeleton, summarize

def analyze_skeleton_skan(skeleton, voxel_size=(1.0, 0.18, 0.18)):
    """
    Analyzes the skeleton using Skan library.

    Parameters:
    - skeleton: 3D NumPy array (boolean), the skeletonized image.
    - voxel_size: tuple of floats, voxel dimensions.

    Returns:
    - num_endpoints: int
    - num_branch_points: int
    - num_branches: int
    - total_branch_length: float
    """
    from skan import Skeleton, summarize

    # Create a Skeleton object
    skeleton_data = Skeleton(skeleton, spacing=voxel_size)
    summary = summarize(skeleton_data)

    # Number of branches
    num_branches = skeleton_data.n_paths

    # Total branch length
    total_branch_length = summary['branch-distance'].sum()

    # Degrees of nodes
    degrees = skeleton_data.degrees

    # Number of endpoints (degree == 1)
    num_endpoints = np.sum(degrees == 1)

    # Number of branch points (degree > 2)
    num_branch_points = np.sum(degrees > 2)

    return num_endpoints, num_branch_points, num_branches, total_branch_length



def extract_astrocyte_features(labeled_astrocytes, gfap_img, voxel_size=(1.0, 0.18, 0.18)):
    import pandas as pd
    import numpy as np
    from skimage.measure import regionprops
    from skimage.morphology import skeletonize_3d
    from skan import Skeleton, summarize

    # Initialize lists to store properties
    labels = []
    volumes = []
    centroids = []
    mean_intensities = []
    num_endpoints_list = []
    num_branch_points_list = []
    num_branches_list = []
    total_branch_lengths = []
    complexities = []
    surface_areas = []

    # Calculate voxel volume
    voxel_volume = voxel_size[0] * voxel_size[1] * voxel_size[2]  # μm³ per voxel

    # Iterate over each astrocyte region
    regions = regionprops(labeled_astrocytes, intensity_image=gfap_img)
    for region in regions:
        label_id = region.label
        labels.append(label_id)

        # Volume in μm³
        volume = region.area * voxel_volume
        volumes.append(volume)

        # Centroid in physical units (μm)
        centroid = (
            region.centroid[0] * voxel_size[0],
            region.centroid[1] * voxel_size[1],
            region.centroid[2] * voxel_size[2]
        )
        centroids.append(centroid)

        # Mean intensity
        mean_intensity = region.mean_intensity
        mean_intensities.append(mean_intensity)

        # Extract the astrocyte mask for this region
        astrocyte_mask = (labeled_astrocytes == label_id)

        # Skeletonize the astrocyte
        skeleton = skeletonize_3d(astrocyte_mask)

        # Check if the skeleton is empty
        if np.sum(skeleton) == 0:
            # Assign zeros or NaN to the skeleton features
            num_endpoints = 0
            num_branch_points = 0
            num_branches = 0
            total_branch_length = 0.0
        else:
            # Analyze the skeleton using Skan
            try:
                num_endpoints, num_branch_points, num_branches, total_branch_length = analyze_skeleton_skan(skeleton, voxel_size)
            except Exception as e:
                print(f"Error analyzing skeleton for label {label_id}: {e}")
                # Assign zeros or NaN to the skeleton features
                num_endpoints = 0
                num_branch_points = 0
                num_branches = 0
                total_branch_length = 0.0

        # Append skeleton features to lists
        num_endpoints_list.append(num_endpoints)
        num_branch_points_list.append(num_branch_points)
        num_branches_list.append(num_branches)
        total_branch_lengths.append(total_branch_length)

        # Calculate 'a', 'b', 'c' for the astrocyte in μm
        bbox = region.bbox  # (min_z, min_y, min_x, max_z, max_y, max_x)
        a = ((bbox[5] - bbox[2]) / 2) * voxel_size[2]  # x-direction (width)
        b = ((bbox[4] - bbox[1]) / 2) * voxel_size[1]  # y-direction (height)
        c = ((bbox[3] - bbox[0]) / 2) * voxel_size[0]  # z-direction (depth)

        # Calculate surface area using an approximate formula (Knud Thomsen's formula)
        p = 1.6075
        surface_area = 4 * np.pi * ((a**p * b**p + a**p * c**p + b**p * c**p) / 3) ** (1/p)
        surface_areas.append(surface_area)

        # Calculate complexity
        complexity = volume / surface_area
        complexities.append(complexity)

    # Create the DataFrame
    astrocyte_df = pd.DataFrame({
        'label': labels,
        'volume': volumes,
        'centroid_z': [c[0] for c in centroids],
        'centroid_y': [c[1] for c in centroids],
        'centroid_x': [c[2] for c in centroids],
        'mean_intensity': mean_intensities,
        'num_endpoints': num_endpoints_list,
        'num_branch_points': num_branch_points_list,
        'num_branches': num_branches_list,
        'total_branch_length': total_branch_lengths,
        'surface_area': surface_areas,
        'complexity': complexities
    })

    return astrocyte_df



from scipy.ndimage import distance_transform_edt
from skimage.filters import threshold_otsu

def segment_plaques(oc_img, e6_img):
    thresh_oc = threshold_otsu(oc_img)
    oc_mask = oc_img > thresh_oc
    thresh_e6 = threshold_otsu(e6_img)
    e6_mask = e6_img > thresh_e6
    plaque_mask = oc_mask | e6_mask
    return plaque_mask

def compute_distance_to_plaques(plaque_mask):
    distance_map = distance_transform_edt(~plaque_mask)
    return distance_map

from skimage.measure import regionprops

def astrocyte_touching_plaques(labeled_astrocytes, plaque_mask):
    touching = []
    for region in regionprops(labeled_astrocytes):
        astrocyte = (labeled_astrocytes == region.label)
        overlap = astrocyte & plaque_mask
        touching.append({'label': region.label, 'touching_plaque': np.any(overlap)})
    touching_df = pd.DataFrame(touching)
    return touching_df


def create_per_astrocyte_dataframe(astrocyte_df, touching_df, distance_map, labeled_astrocytes):
    # Compute mean distance from plaques for each astrocyte
    mean_distances = []
    for label_id in astrocyte_df['label']:
        astrocyte = (labeled_astrocytes == label_id)
        distances = distance_map[astrocyte]
        mean_distances.append(distances.mean())
    astrocyte_df['mean_distance_to_plaque'] = mean_distances
    # Merge with touching information
    astrocyte_df = astrocyte_df.merge(touching_df, on='label')
    return astrocyte_df

def create_per_image_dataframe(astrocyte_df, nuclei_df, plaque_mask):
    total_astrocytes = len(astrocyte_df)
    mean_nuclei_distance = nuclei_df['distance_to_plaque'].mean()
    mean_process_distance = astrocyte_df['mean_distance_to_plaque'].mean()
    mean_astrocyte_size = astrocyte_df['volume'].mean()
    plaque_volume = plaque_mask.sum()
    
    image_stats = {
        'total_astrocytes': total_astrocytes,
        'mean_nuclei_distance': mean_nuclei_distance,
        'mean_process_distance': mean_process_distance,
        'mean_astrocyte_size': mean_astrocyte_size,
        'plaque_volume': plaque_volume
    }
    image_df = pd.DataFrame([image_stats])
    return image_df

import pandas as pd
from skimage.measure import regionprops


all_astrocyte_data = []
all_image_data = []

for gfap_file, nuclei_file, oc_file, e6_file in file_pairs:
    # Load images
    gfap_img = tifffile.imread(gfap_file)
    nuclei_img = tifffile.imread(nuclei_file)
    oc_img = tifffile.imread(oc_file)
    e6_img = tifffile.imread(e6_file)

    # Align images
    max_slices = max(gfap_img.shape[0], oc_img.shape[0], e6_img.shape[0])

    def pad_image(image, max_slices):
        num_slices = image.shape[0]
        if num_slices < max_slices:
            padding = ((0, max_slices - num_slices), (0, 0), (0, 0))
            return np.pad(image, padding, mode='constant', constant_values=0)
        else:
            return image

    gfap_img = pad_image(gfap_img, max_slices)
    oc_img = pad_image(oc_img, max_slices)
    e6_img = pad_image(e6_img, max_slices)
    nuclei_img = pad_image(nuclei_img, max_slices)
    
    # Get target shape from GFAP or Nuclei images
    target_shape = gfap_img.shape  # Assuming GFAP images are 1024x1024
    
    # Pad OC image
    oc_img = pad_image_to_target_size(oc_img, target_shape)
    
    # Pad 6E10 image
    e6_img = pad_image_to_target_size(e6_img, target_shape)
    
    # Verify shapes
    print("GFAP image shape:", gfap_img.shape)
    print("Nuclei image shape:", nuclei_img.shape)
    print("OC image shape (padded):", oc_img.shape)
    print("6E10 image shape (padded):", e6_img.shape)
    
    # Visualize (optional)
    
    # After segmenting the nuclei and plaques
    labeled_nuclei = segment_nuclei(nuclei_img)
    plaque_mask = segment_plaques(oc_img, e6_img)
    oc_mask = segment_oc_objects(oc_img)
    plot_max_projection(oc_mask, 'Plaque Mask')

    # Identify nuclei touching the plaque
    nuclei_to_remove = get_nuclei_touching_plaque(labeled_nuclei, plaque_mask)

    # Remove nuclei touching the plaque
    labeled_nuclei_clean = remove_nuclei_touching_plaque(labeled_nuclei, nuclei_to_remove)

    markers = labeled_nuclei_clean
    num_nuclei_before = np.max(labeled_nuclei)
    num_nuclei_to_remove = len(nuclei_to_remove)

    print(f"Number of nuclei before removal: {num_nuclei_before}")
    print(f"Number of nuclei to remove: {num_nuclei_to_remove}")

    nuclei_df = measure_nuclei_volumes(labeled_nuclei_clean)

    # Calculate volume percentiles
    volume_min, volume_max = get_volume_percentiles(nuclei_df)

    # Identify nuclei within the volume range
    valid_nuclei_labels = filter_nuclei_by_volume(nuclei_df, volume_min, volume_max)

    # Remove nuclei outside the volume range
    labeled_nuclei_filtered = remove_nuclei_outside_volume_range(labeled_nuclei_clean, valid_nuclei_labels)

    
    # Proceed with the rest of the analysis using labeled_nuclei_clean
    # Use labeled_nuclei_clean as markers in the watershed segmentation
    voxel_size = (1.0, 0.18, 0.18)  # depth (z), height (y), width (x) in μm

    # Segment GFAP and merge small segments
    labeled_gfap = segment_gfap(gfap_img, labeled_nuclei_filtered, oc_mask, voxel_size=voxel_size)

    # Assuming you have labeled_astrocytes and labeled_nuclei
    voxel_size = (1.0, 0.18, 0.18)  # Replace with your actual voxel size
    max_volume = 2500  # Maximum allowed volume in μm³

    # Split large astrocyte segments
    #labeled_astrocytes = split_large_astrocytes(labeled_gfap, labeled_nuclei_filtered, voxel_size, max_volume)

    # Remove small astrocyte segments
    astrocyte_vols = astrocyte_volume(labeled_gfap, voxel_size)
    
    #descending order of volume
    astrocyte_vols = astrocyte_vols.sort_values(by='volume', ascending=False)

    print(astrocyte_vols)

    # Filter astrocytes by volume


    
    # Proceed with feature extraction using the labeled_astrocytes
    #astrocyte_df = extract_astrocyte_features(labeled_astrocytes, gfap_img)

    #astrocyte_mask = find_astrocytes_touching_nuclei(labeled_gfap, labeled_nuclei)
    #labeled_astrocytes = close_astrocyte_objects(astrocyte_mask)

    plot_max_projection(gfap_img, 'GFAP Max Projection')
    plot_max_projection(labeled_nuclei_filtered, 'Nuclei Max Projection')

    #plot max projection of astrocytes
    plot_max_projection(labeled_gfap, 'Astrocytes Max Projection')

    
    visualize_astrocytes_max_projection(labeled_gfap, title='Astrocyte Segmentation - Max Projection')

    #plot max projection of astrocytes
    #plot_max_projection(labeled_astrocytes, 'Astrocytes Max Projection with filtering')
    
    # Feature Extraction
    astrocyte_df = extract_astrocyte_features(labeled_gfap, gfap_img)
    
    # Plaque Segmentation and Distance Map
    plaque_mask = segment_plaques(oc_img, e6_img)
    distance_map = compute_distance_to_plaques(plaque_mask)
    
    # Nuclei Distance to Plaques
    nuclei_props = regionprops(labeled_nuclei)
    nuclei_data = []
    for prop in nuclei_props:
        coord = prop.centroid
        z, y, x = int(coord[0]), int(coord[1]), int(coord[2])

        # Ensure indices are within bounds
        if z >= distance_map.shape[0] or y >= distance_map.shape[1] or x >= distance_map.shape[2]:
            print(f"Coordinate ({z}, {y}, {x}) is out of bounds.")
            continue

        distance = distance_map[z, y, x]
        nuclei_data.append({'label': prop.label, 'distance_to_plaque': distance})
    nuclei_df = pd.DataFrame(nuclei_data)
    
    # Astrocyte Processes Touching Plaques
    touching_df = astrocyte_touching_plaques(labeled_gfap, plaque_mask)
    
    # Update Astrocyte Dataframe
    astrocyte_df = create_per_astrocyte_dataframe(astrocyte_df, touching_df, distance_map, labeled_gfap)
    astrocyte_df['image_file'] = gfap_file  # Add image reference
    all_astrocyte_data.append(astrocyte_df)

    

    
    # Create Per-Image Dataframe
    image_df = create_per_image_dataframe(astrocyte_df, nuclei_df, plaque_mask)
    image_df['image_file'] = gfap_file
    all_image_data.append(image_df)
    
# Concatenate all dataframes
final_astrocyte_df = pd.concat(all_astrocyte_data, ignore_index=True)
final_image_df = pd.concat(all_image_data, ignore_index=True)

# Save dataframes to CSV
final_astrocyte_df.to_csv('astrocyte_data.csv', index=False)
#final_image_df.to_csv('image_data.csv', index=False)





